In [190]:
#Collect all the games with stats from 2008
import pandas as pd
import numpy as np
pt = pd.read_csv("../data/B1_FavGames.csv")
KBBP = pd.read_csv("../data/B3_KPBPIBRP.csv").fillna(0)
pt = pt.loc[(pt['Year']>=2008) & (pt['Round'] !='PI')]
pt['Round'] = pt['Round'].astype(int)
pt = pt.iloc[:,0:10]
pt['PFSeed'] = pt['AFSeed']
pt['PFTeam'] = pt['AFTeam']
pt['PFScore'] = pt['AFScore']
pt['PUSeed'] = pt['AUSeed']
pt['PUTeam'] = pt['AUTeam']
pt['PUScore'] = pt['AUScore']
ptf = pt.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
ptb = ptf.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')


In [191]:
# Build a model to predict Final Scores
ptn = ptb.select_dtypes(exclude=['object'])
expl = ptn.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore'],axis=1)
respF = ptn[['PFScore']]
respU = ptn[['PUScore']]
from sklearn.linear_model import ElasticNet
enet_F = ElasticNet(alpha=.284,l1_ratio=.5)
enet_F.fit(expl,respF)
enet_U = ElasticNet(alpha=.39,l1_ratio=.5)
enet_U.fit(expl,respU)
expl.columns


Index(['Year', 'Round', 'Game', 'PFSeed', 'PUSeed', 'AdjEM_x', 'AdjO_x',
       'AdjD_x', 'AdjT_x', 'SOSEM_x', 'SOSO_x', 'SOSD_x', 'BPIO_x', 'BPID_x',
       'W_x', 'L_x', 'Pts_x', 'Opp_x', 'MOV_x', 'SOS_x', 'OSRS_x', 'DSRS_x',
       'SRS_x', 'PASE_x', 'AdjEM_y', 'AdjO_y', 'AdjD_y', 'AdjT_y', 'SOSEM_y',
       'SOSO_y', 'SOSD_y', 'BPIO_y', 'BPID_y', 'W_y', 'L_y', 'Pts_y', 'Opp_y',
       'MOV_y', 'SOS_y', 'OSRS_y', 'DSRS_y', 'SRS_y', 'PASE_y'],
      dtype='object')

In [192]:
# Pick a random Year
from random import choice
p_year = choice([i for i in range(2008,2021) if i not in [2020]])
p_year

2009

In [193]:
# Start with p_year bracket with stats
pty = pt[pt['Year']==p_year]
pty.index = pty.Game

In [194]:
# Build Round 1
pty1 = pty[pty['Round']==1]
pty1 = pty1.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pty1 = pty1.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')
pty1n = pty1.select_dtypes(exclude=['object'])
expl_R1 = pty1n.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore'],axis=1)
respF_R1 = enet_F.predict(expl_R1)
respU_R1 = enet_U.predict(expl_R1)
for x in range(1,33):
    pty.loc[x,'PFScore'] = np.around(respF_R1[x-1],3)
    pty.loc[x,'PUScore'] = np.around(respU_R1[x-1],3)
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],10,0)


C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)


In [195]:
# Build Round 2
# pull the predicted winners from round 1 
for x in range(1,17):
    pty.loc[x+32,'PFSeed']=min((pty.loc[2*x-1,'PWSeed']),(pty.loc[2*x,'PWSeed']))
    pty.loc[x+32,'PUSeed']=max((pty.loc[2*x-1,'PWSeed']),(pty.loc[2*x,'PWSeed']))
    pty.loc[x+32,'PFTeam']=str(np.where((pty.loc[2*x-1,'PWSeed'])<(pty.loc[2*x,'PWSeed']),pty.loc[2*x-1,'PWTeam'],pty.loc[2*x,'PWTeam']))
    pty.loc[x+32,'PUTeam']=str(np.where((pty.loc[2*x-1,'PWSeed'])>(pty.loc[2*x,'PWSeed']),pty.loc[2*x-1,'PWTeam'],pty.loc[2*x,'PWTeam']))  
    
# Now Mimic Round 1
pty2 = pty[pty['Round']==2]
pty2 = pty2.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pty2 = pty2.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')
pty2n = pty2.select_dtypes(exclude=['object'])
expl_R2 = pty2n.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore','AWSeed','PWSeed','ESPN'],axis=1)
respF_R2 = enet_F.predict(expl_R2)
respU_R2 = enet_U.predict(expl_R2)
for x in range(33,49):
    pty.loc[x,'PFScore'] = np.around(respF_R1[x-33],3)
    pty.loc[x,'PUScore'] = np.around(respU_R1[x-33],3)
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],20,0)

C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [196]:
# Build Round 3
# pull the predicted winners from round 2 
for x in range(1,9):
    pty.loc[x+48,'PFSeed']=min((pty.loc[2*x+31,'PWSeed']),(pty.loc[2*x+32,'PWSeed']))
    pty.loc[x+48,'PUSeed']=max((pty.loc[2*x+31,'PWSeed']),(pty.loc[2*x+32,'PWSeed']))
    pty.loc[x+48,'PFTeam']=str(np.where((pty.loc[2*x+31,'PWSeed'])<(pty.loc[2*x+32,'PWSeed']),pty.loc[2*x+31,'PWTeam'],pty.loc[2*x+32,'PWTeam']))
    pty.loc[x+48,'PUTeam']=str(np.where((pty.loc[2*x+31,'PWSeed'])>(pty.loc[2*x+32,'PWSeed']),pty.loc[2*x+31,'PWTeam'],pty.loc[2*x+32,'PWTeam']))  
# Now Mimic Round 2
pty3 = pty[pty['Round']==3]
pty3 = pty3.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pty3 = pty3.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')
pty3n = pty3.select_dtypes(exclude=['object'])
expl_R3 = pty3n.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore','AWSeed','PWSeed','ESPN'],axis=1)
respF_R3 = enet_F.predict(expl_R3)
respU_R3 = enet_U.predict(expl_R3)
for x in range(49,57):
    pty.loc[x,'PFScore'] = np.around(respF_R1[x-49],3)
    pty.loc[x,'PUScore'] = np.around(respU_R1[x-49],3)
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],40,0)   


C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [197]:
# Build Round 4
# pull the predicted winners from round 3 
for x in range(1,5):
    pty.loc[x+56,'PFSeed']=min((pty.loc[2*x+47,'PWSeed']),(pty.loc[2*x+48,'PWSeed']))
    pty.loc[x+56,'PUSeed']=max((pty.loc[2*x+47,'PWSeed']),(pty.loc[2*x+48,'PWSeed']))
    pty.loc[x+56,'PFTeam']=str(np.where((pty.loc[2*x+47,'PWSeed'])<(pty.loc[2*x+48,'PWSeed']),pty.loc[2*x+47,'PWTeam'],pty.loc[2*x+48,'PWTeam']))
    pty.loc[x+56,'PUTeam']=str(np.where((pty.loc[2*x+47,'PWSeed'])>(pty.loc[2*x+48,'PWSeed']),pty.loc[2*x+47,'PWTeam'],pty.loc[2*x+48,'PWTeam']))  
# Now Mimic Round 3
pty4 = pty[pty['Round']==4]
pty4 = pty4.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pty4 = pty4.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')
pty4n = pty4.select_dtypes(exclude=['object'])
expl_R4 = pty4n.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore','AWSeed','PWSeed','ESPN'],axis=1)
respF_R4 = enet_F.predict(expl_R4)
respU_R4 = enet_U.predict(expl_R4)
for x in range(57,61):
    pty.loc[x,'PFScore'] = np.around(respF_R1[x-57],3)
    pty.loc[x,'PUScore'] = np.around(respU_R1[x-57],3)
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],80,0)   

C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [198]:
# Build Round 5
# pull the predicted winners from round 4 
for x in range(1,3):
    pty.loc[x+60,'PFSeed']=min((pty.loc[2*x+55,'PWSeed']),(pty.loc[2*x+56,'PWSeed']))
    pty.loc[x+60,'PUSeed']=max((pty.loc[2*x+55,'PWSeed']),(pty.loc[2*x+56,'PWSeed']))
    pty.loc[x+60,'PFTeam']=str(np.where((pty.loc[2*x+55,'PWSeed'])<(pty.loc[2*x+56,'PWSeed']),pty.loc[2*x+55,'PWTeam'],pty.loc[2*x+56,'PWTeam']))
    pty.loc[x+60,'PUTeam']=str(np.where((pty.loc[2*x+55,'PWSeed'])>(pty.loc[2*x+56,'PWSeed']),pty.loc[2*x+55,'PWTeam'],pty.loc[2*x+56,'PWTeam']))  
# Now Mimic Round 4
pty5 = pty[pty['Round']==5]
pty5 = pty5.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pty5 = pty5.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')
pty5n = pty5.select_dtypes(exclude=['object'])
expl_R5 = pty5n.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore','AWSeed','PWSeed','ESPN'],axis=1)
respF_R5 = enet_F.predict(expl_R5)
respU_R5 = enet_U.predict(expl_R5)
for x in range(61,63):
    pty.loc[x,'PFScore'] = np.around(respF_R1[x-61],3)
    pty.loc[x,'PUScore'] = np.around(respU_R1[x-61],3)
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],160,0)   

C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [199]:
# Build Round 6
# pull the predicted winners from round 5
for x in range(1,2):
    pty.loc[x+62,'PFSeed']=min((pty.loc[2*x+59,'PWSeed']),(pty.loc[2*x+60,'PWSeed']))
    pty.loc[x+62,'PUSeed']=max((pty.loc[2*x+59,'PWSeed']),(pty.loc[2*x+60,'PWSeed']))
    pty.loc[x+62,'PFTeam']=str(np.where((pty.loc[2*x+59,'PWSeed'])<(pty.loc[2*x+60,'PWSeed']),pty.loc[2*x+59,'PWTeam'],pty.loc[2*x+60,'PWTeam']))
    pty.loc[x+62,'PUTeam']=str(np.where((pty.loc[2*x+59,'PWSeed'])>(pty.loc[2*x+60,'PWSeed']),pty.loc[2*x+59,'PWTeam'],pty.loc[2*x+60,'PWTeam'])) 
# Now Mimic Round 5
pty6 = pty[pty['Round']==6]
pty6 = pty6.merge(KBBP, left_on=['Year','PFTeam'],right_on=['Year','Team'],how='left')
pty6 = pty6.merge(KBBP, left_on=['Year','PUTeam'],right_on=['Year','Team'],how='left')
pty6n = pty6.select_dtypes(exclude=['object'])
expl_R6 = pty6n.drop(['AFScore','AUScore', 'AFSeed', 'AUSeed','PFScore','PUScore','AWSeed','PWSeed','ESPN'],axis=1)
respF_R6 = enet_F.predict(expl_R6)
respU_R6 = enet_U.predict(expl_R6)
for x in range(63,64):
    pty.loc[x,'PFScore'] = np.around(respF_R1[x-63],3)
    pty.loc[x,'PUScore'] = np.around(respU_R1[x-63],3)
    pty.loc[x,'AWSeed'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFSeed'],pty.loc[x,'AUSeed'])
    pty.loc[x,'AWTeam'] = np.where(pty.loc[x,'AFScore']>=pty.loc[x,'AUScore'],pty.loc[x,'AFTeam'],pty.loc[x,'AUTeam'])
    pty.loc[x,'PWSeed'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFSeed'],pty.loc[x,'PUSeed'])
    pty.loc[x,'PWTeam'] = np.where(pty.loc[x,'PFScore']>=pty.loc[x,'PUScore'],pty.loc[x,'PFTeam'],pty.loc[x,'PUTeam'])
    pty.loc[x,'ESPN'] = np.where(pty.loc[x,'AWTeam']==pty.loc[x,'PWTeam'],320,0)  

C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
C:\Users\harmk\AppData\Local\Programs\Python\Python39\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [200]:
print(p_year)
print(pty.ESPN.sum())
pty.to_csv('../data/tal.csv')

2009
620.0
